In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import src.graphinglib as gl
from collections import namedtuple
import pyregion

from src.hdu.tesseract import Tesseract
from src.hdu.cubes.cube import Cube
from src.hdu.maps.map import Map
from src.hdu.maps.grouped_maps import GroupedMaps

In [ ]:
Info = namedtuple("Info", ["name", "cube_filename", "gaussian_index", "relative_error_threshold"])

files = [
    Info("nii_1", "nii_1_binned_3x3.fits", 1, 0.0016),
    Info("nii_2", "nii_2_binned_4x4.fits", 1, 0.003),
    Info("oiii_1", "oiii_1_binned_3x3.fits", 0, 0.09),
    Info("oiii_2", "oiii_2_binned_4x4.fits", 0, 0.03),
    Info("sii_1", "sii_1_binned_3x3.fits", [0, 1], 0.009),
    Info("sii_2", "sii_2_binned_4x4.fits", [0, 1], 0.016),
    Info("ha_1", "ha_1_binned_3x3.fits", "variable", 0.015),
    Info("ha_2", "ha_2_binned_4x4.fits", 0, 0.005),
]

def reject_outliers(data):
    # Consider 3 sigma values as outliers
    return abs(data - np.nanmedian(data)) < 3 * np.nanstd(data)

def make_fit_histogram(data: np.ndarray, bin_width: float) -> list[gl.Histogram, gl.Curve]:
    data = data.flatten()
    data = np.sort(data[~np.isnan(data)])

    # Create the array of bins, considering that the upper bound is cropped lower to match the bin width
    bins = np.arange(np.nanmin(data), np.nanmax(data) + bin_width*0.99, bin_width)
    hist = np.histogram(data, bins=bins)[0]

    current_plots = [
        gl.Histogram(
            data=data,
            face_color="dimgray",
            edge_color="none",
            normalize=False,
            alpha=1,
            number_of_bins=bins,
            show_params=False
        ),
    ]
    scat = gl.Scatter(
        (bins + bin_width/2)[:-1],
        hist,
    )

    max_x = scat.x_data[np.argmax(hist)]
    fit = gl.FitFromFunction(
        function=lambda x, A, mu, sigma: A* np.exp(-(x - mu)**2 / (2*sigma**2)),
        curve_to_be_fit=scat,
        guesses=[
            np.max(hist)*max_x,
            max_x,
            np.nanstd(data)
        ],
        color="red",
    )
    fit.label = rf"$m={fit.parameters[1]:.2f}$" + "\n" + rf"$s={abs(fit.parameters[2]):.2f}$"
    current_plots.append(fit)

    return current_plots

# Unfiltered statistics

#### Centroid maps creation

In [ ]:
for info in files:
    cube = Cube.load(f"data/orion/data_cubes/binned/{info.cube_filename}")
    tess = Tesseract.load(f"data/orion/fits/{info.name}.fits")
    reg = pyregion.open(f"data/orion/fp_confidence_regions/{info.name}.reg")
    if isinstance(info.gaussian_index, int):
        centroids = tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean[info.gaussian_index]
        centroids = centroids.get_masked_region(reg)
        centroids = centroids.mask((centroids.uncertainties / centroids.data) < info.relative_error_threshold)
        centroids = centroids.mask(reject_outliers(centroids.data))
        centroids = (centroids - 1) * cube.header["CHAN_KMS"] * 1000 + cube.header["VR_CH1"] * 1000
        centroids /= 1000  # convert to km/s

    else:  # sii/ha_1
        centroids_maps = []
        if info.name.startswith("sii"):
            mean_maps = tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean
        else:  # ha_1 case
            filtered_tess = tess.filter(slice(18, 25))  # take only the desired components
            mean_maps = filtered_tess.to_grouped_maps(["amplitude", "mean", "fwhm_L", "fwhm_G"]).mean[1:3]
        for centroid in mean_maps:
            centroid = centroid.get_masked_region(reg)
            centroid = centroid.mask((centroid.uncertainties / centroid.data) < info.relative_error_threshold)
            centroid = centroid.mask(reject_outliers(centroid.data))
            centroid_speed = (centroid - 1) * cube.header["CHAN_KMS"] * 1000 + cube.header["VR_CH1"] * 1000
            centroid_speed /= 1000  # convert to km/s
            centroids_maps.append(centroid_speed)
        centroids = GroupedMaps([("centroids", centroids_maps)])

    centroids.save(f"data/orion/centroids/maps/{info.name}_centroids.fits")

#### Separate unfiltered histograms

In [ ]:
for info in files:
    if info.name in ["sii_1", "sii_2", "ha_1"]:
        centroids = GroupedMaps.load(f"data/orion/centroids/maps/{info.name}_centroids.fits").centroids
        centroid_data = np.stack([centroid.data for centroid in centroids], axis=0)
        bin_width = np.nanmedian(centroids[0].uncertainties)

    else:
        centroids = Map.load(f"data/orion/centroids/maps/{info.name}_centroids.fits")
        centroid_data = centroids.data
        bin_width = np.nanmedian(centroids.uncertainties)

    plots = make_fit_histogram(centroid_data, bin_width)
    fig = gl.SmartFigure(
        elements=plots,
        title=f"Centroid {info.name}",
        x_label="Centroid velocity [km s$^{-1}$]",
    ).show()


#### Merged unfiltered histograms

In [ ]:
maps = {}

for info in files:
    if info.name in ["sii_1", "sii_2", "ha_1"]:
        centroids = GroupedMaps.load(f"data/orion/centroids/maps/{info.name}_centroids.fits").centroids
        centroid_data = np.stack([centroid.data for centroid in centroids], axis=0)
        centroid_uncertainties = np.stack([centroid.uncertainties for centroid in centroids], axis=0)

    else:
        centroids = Map.load(f"data/orion/centroids/maps/{info.name}_centroids.fits")
        centroid_data = centroids.data
        centroid_uncertainties = centroids.uncertainties

    key = info.name.split("_")[0]
    if key not in maps:
        maps[key] = {"data" : [], "uncertainties": []}
    maps[key]["data"].append(centroid_data)
    maps[key]["uncertainties"].append(centroid_uncertainties)

for key, value in maps.items():
    # if key != "sii": continue
    merged_data = np.concatenate([v.flatten() for v in value["data"]])
    merged_uncertainties = np.concatenate([v.flatten() for v in value["uncertainties"]])
    bin_width = np.nanmedian(merged_uncertainties)
    plots = make_fit_histogram(merged_data, bin_width)
    fig = gl.SmartFigure(
        elements=plots,
        title=f"Centroid {key}",
        x_label="Centroid velocity [km s$^{-1}$]",
    ).show()

#### Philippe fun...

In [ ]:
from scipy.constants import c

lambda_channel_1 = 6561.0
lambda_ref = 6562.7797852

print(f"{(lambda_channel_1 - lambda_ref) / lambda_ref * c / 1000} km/s")


In [ ]:
# OIII MADNESS
spec_1 = Cube.load("data/orion/data_cubes/binned/oiii_1_binned_3x3.fits")[:, 100, 100].plot
spec_2 = Cube.load("data/orion/data_cubes/binned/oiii_2_binned_4x4.fits")[:, 60, 60].plot

spec_1.label = "[OIII] Field 1"
spec_2.label = "[OIII] Field 2"
spec_2.color = gl.get_color(color_number=2)

gl.SmartFigure(
    1,
    2,
    elements=[
        spec_1,
        spec_2,
    ],
    size=(8, 5)
).set_grid().show().save("oiii_spectrums.pdf")

# Filtered statistics